##import packages

In [289]:
import csv
import utm
import pandas as pd
import datetime as dt
from datetime import datetime
import geopandas as gpd
import numpy as np
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [290]:
dp_start = dt.time(7,0,0)
dp_end = dt.time(9,0,0)
pk_start = dt.time(16,0,0)
pk_end = dt.time(18,0,0)

In [291]:
yellow_trip = pd.read_csv('/Users/mac/Desktop/yellow-trip-new03.csv')

yellow_trip['dp_datetime'] = pd.to_datetime(yellow_trip['tpep_dropoff_datetime'])
yellow_trip['pk_datetime'] = pd.to_datetime(yellow_trip['tpep_pickup_datetime'])
yellow_trip['dp_time'] = yellow_trip['dp_datetime'].dt.time
yellow_trip['pk_time'] = yellow_trip['pk_datetime'].dt.time

##select weekdays

In [292]:
yellow_trip['weekday'] = yellow_trip['dp_datetime'].apply(lambda x: x.weekday())
yp_weekdays_only = yellow_trip[yellow_trip['weekday'] < 5 ]

##select peak hours

In [293]:
yp_dp = yp_weekdays_only[yp_weekdays_only['dp_time'].between(dp_start,dp_end)]
yp_pk = yp_weekdays_only[yp_weekdays_only['pk_time'].between(pk_start,pk_end)]

##get SCP-station nad 

In [294]:
station_nad = gpd.read_file('/Users/mac/Desktop/SCP_Station/SCP_station_nad.shp')
station = []
for index, row in station_nad.iterrows():
    for st in list(row['geometry'].coords):
        station.append(st)

##find dp_point within each station

In [295]:
demand_station = []
for st_candidate in station:
    dt_temp = []
    st_buffer = Point(st_candidate).buffer(1000)
    for dp in yp_dp.index:
        x = utm.from_latlon(yp_dp.loc[dp,'Dropoff_latitude'],yp_dp.loc[dp,'Dropoff_longitude'])[0:2]
        if st_buffer.contains(Point(x)):
            dt_temp.append(dp)
    demand_station.append(dt_temp)

##convert dataframe to csv of station_demand

In [488]:
df1 = yp_dp[yp_dp.index.isin(demand_station[1])]
df1.to_csv('/Users/mac/Desktop/df1.csv')

In [490]:
def dist(n1, n2):
    x1, y1 = utm.from_latlon(df1.iloc[n1]['Pickup_latitude'],df1.iloc[n1]['Pickup_longitude'])[0:2]
    x2, y2 = utm.from_latlon(df1.iloc[n2]['Pickup_latitude'],df1.iloc[n2]['Pickup_longitude'])[0:2]
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [491]:
def capa(n1, n2):
    c1 = df1.iloc[n1]['Passenger_count']
    c2 = df1.iloc[n2]['Passenger_count']
    if c1 + c2 > 4:
        total = float('inf')
    else:
        total = 1
    return total

In [492]:
def time(n1, n2):
    t1 = str(df1.iloc[n1]['pk_time'])
    t2 = str(df1.iloc[n2]['pk_time'])
    FMT = '%H:%M:%S'
    tdelta = datetime.strptime(t2, FMT) - datetime.strptime(t1, FMT)
    mindelta = tdelta.seconds/60
    if mindelta > 10:
        m = float('inf')
    else:
        m = mindelta
    return m

In [493]:
def cost(n1, n2):
    cost = dist(n1, n2) * capa(n1, n2) * time(n1, n2)
    return cost

In [494]:
matrix_1 = np.zeros((len(demand_df1.index), len(demand_df1.index)))

i = 0
while i < len(demand_df1.index):
    j = 0
    while j < len(demand_df1.index):
        matrix_1[i][j] = cost(i,j)
        #matrix_1[i][j] = 2
        j = j+1
    i = i+1